In [1]:
# Module Importations
import datetime
from matplotlib import pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split

In [2]:
# Project Module Importations
from data_analytics.src.data import cosmos_client_manager
from data_analytics.src.data import load_data
from data_analytics.src.data import test_unit
from data_analytics.src.features import data_munging

In [4]:
# Load data from local store
df_new = load_data.load_dash_dataframe()

AttributeError: module 'data_analytics.src.data.load_data' has no attribute 'load_dash_dataframe'